In [35]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import misc as ms

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [108]:
reload(dl)
reload(ms)

<module 'fitter.misc' from '../fitter\misc.pyc'>

In [109]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info(whose='mine')
other_results = data_loader.get_fit_info(whose='others')

In [110]:
 fit_results

OrderedDict([('ma-ratio_FKFK_nnlo',
              {'A_a': '-1.9(1.1)',
               'A_k': '3.4(2.3)',
               'A_loga': nan,
               'A_p': '-1.0(1.6)',
               'L_4': '0.0051(42)',
               'L_5': '0.0015(13)',
               'Q': 0.0011749638684156554,
               'Unnamed: 0': 24L,
               'chi2/df': 2.4229518230984435,
               'delta_su2': '-0.00243(20)',
               'fit': '1.1788(30)',
               'logGBF': 53.78529931609321,
               'name': 'ma-ratio_FKFK_nnlo',
               'vol corr': 0L}),
             ('ma-ratio_FKFK_nnlo_FV',
              {'A_a': '-1.5(1.2)',
               'A_k': '2.7(2.5)',
               'A_loga': nan,
               'A_p': '-0.6(1.7)',
               'L_4': '0.0037(37)',
               'L_5': '0.0014(11)',
               'Q': 0.002865049435100901,
               'Unnamed: 0': 28L,
               'chi2/df': 2.2519290256597526,
               'delta_su2': '-0.00245(20)',
               'fit': 

In [111]:
temp_results = {model : fit_results[model] for model in fit_results.keys()}

In [112]:
temp_results = {}
for model in fit_results.keys():
    if '_FKFpi_' in model:
        temp_results[model] = fit_results[model]
        
temp_results = fit_results

In [113]:
import scipy.stats as stats

def bma(fit_results):
    # read Bayes Factors
    logGBF_list = [fit_results[model]['logGBF'] for model in fit_results.keys()]
    
    # initiate a bunch of parameters
    FK_per_Fpi = 0
    FK_per_Fpi_list = []
    FK_per_Fpi_dict = dict()
    
    # weights
    w_lst = []
    wd = dict()
    
    # p. dist. fcn
    pdf = 0
    pdfdict = dict()
    
    # c. dist. fcn.
    cdf = 0
    cdfdict = dict()
    
    
    # for plotting
    x = np.linspace(1.16,1.20,2000)
    
    for model in fit_results.keys():
        r = gv.gvar(fit_results[model]['fit'])
        FK_per_Fpi_dict[model] = r
        
        w = 1/sum(np.exp(np.array(logGBF_list)-fit_results[model]['logGBF']))
        sqrtw = np.sqrt(w) # sqrt scales the std dev correctly
        wd[model] = w
        w_lst.append(w)
        
        FK_per_Fpi += gv.gvar(w*r.mean,sqrtw*r.sdev)
        FK_per_Fpi_list.append(r.mean)
        
        p = stats.norm.pdf(x,r.mean,r.sdev)
        pdf += w*p
        pdfdict[model] = w*p
        
        
        c = stats.norm.cdf(x,r.mean,r.sdev)
        cdf += w*c
        cdfdict[model] = w*c
        
    FK_per_Fpi_list = np.array(FK_per_Fpi_list)
    w_lst = np.array(w_lst)
    
    plot_params = {'x':x, 'pdf':pdf, 'pdfdict':pdfdict, 'cdf':cdf, 'cdfdict':cdfdict}
    return plot_params

params = bma(temp_results)

In [116]:
def plot_histogram(fit_results, plot_params):
    gr = 1.618034333
    fs2_base = 3.50394
    plt_axes = [0.14,0.165,0.825,0.825]
    fig_size2 = (fs2_base,fs2_base/gr)
    lw = 0.5
    fs_l = 15
    fs_xy = 24
    ts = 15

    x = plot_params['x']
    ysum = plot_params['pdf']

    ydict = plot_params['pdfdict']
    cdf = plot_params['cdf']

    # '-','--','-.',':'
    # #ec5d57 #70bf41 #51a7f9

    fig = plt.figure('result histogram')#,figsize=fig_size2)
    ax = plt.axes(plt_axes)
    ax.fill_between(x=x,y1=ysum,facecolor='#b36ae2',edgecolor='black',alpha=0.4,label='model average')
    # get 95% confidence
    lidx95 = abs(cdf-0.025).argmin()
    uidx95 = abs(cdf-0.975).argmin()
    ax.fill_between(x=x[lidx95:uidx95],y1=ysum[lidx95:uidx95],facecolor='#b36ae2',edgecolor='black',alpha=0.4)
    # get 68% confidence
    lidx68 = abs(cdf-0.158655254).argmin()
    uidx68 = abs(cdf-0.841344746).argmin()
    ax.fill_between(x=x[lidx68:uidx68],y1=ysum[lidx68:uidx68],facecolor='#b36ae2',edgecolor='black',alpha=0.4)
    # plot black curve over
    ax.errorbar(x=[x[lidx95],x[lidx95]],y=[0,ysum[lidx95]],color='black',lw=lw)
    ax.errorbar(x=[x[uidx95],x[uidx95]],y=[0,ysum[uidx95]],color='black',lw=lw)
    ax.errorbar(x=[x[lidx68],x[lidx68]],y=[0,ysum[lidx68]],color='black',lw=lw)
    ax.errorbar(x=[x[uidx68],x[uidx68]],y=[0,ysum[uidx68]],color='black',lw=lw)

    ax.errorbar(x=x,y=ysum,ls='-',color='black',lw=lw)

    for a in ydict.keys():
        ax.plot(x,ydict[a])

    leg = ax.legend(fontsize=fs_l, edgecolor='k',fancybox=False)
    ax.set_ylim(bottom=0)
    #ax.set_xlim([1.225,1.335])
    ax.set_xlabel('$F_K/F_\pi$', fontsize=fs_xy)
    frame = plt.gca()
    frame.axes.get_yaxis().set_visible(False)
    ax.xaxis.set_tick_params(labelsize=ts,width=lw)

    # legend line width
    [ax.spines[key].set_linewidth(lw) for key in ax.spines]
    leg.get_frame().set_linewidth(lw)


    #plt.show()
    plt.close()


plot_histogram(temp_results, params)